In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data/Training_Data/alldata_tokenized.csv")

In [7]:
df.head()

,Unnamed: 0,label,uuid,title,text,pre_processed_text
0,0,Entertainment,afc6326a4fafdd25b0470d6e3505234ddedda086,Tracy Morgan performs first stand-up show sinc...,Tracy Morgan performs first stand-up show sinc...,"['tracy', 'morgan', 'performs', 'first', 'stan..."
1,1,Entertainment,ac0f86c24d1a083941f7499166aeb8992157e06c,Esquire's Sexiest Woman Alive is the sultry Em...,SUBSCRIBE FOLLOW US © 2015 Cox Media Group . B...,"['subscribe', 'follow', 'us', 'two thousand an..."
2,2,Entertainment,09b3e01b14eaa909225806ee4bf95ac297b66e00,President Barack Obama interviews author Maril...,FCC Information President Barack Obama intervi...,"['fcc', 'information', 'president', 'barack', ..."
3,3,Entertainment,37b6285ff56465ce25602ebf4a2f74f331a70811,"I will perform at night clubs, shows – Pastor ...",Source: Ghana | Hitz FM | Euphemia Akpalu Date...,"['source', 'ghana', 'hitz', 'fm', 'euphemia', ..."
4,4,Entertainment,150c5db5233b5cbd5eb00dcfce68dd88f21f4287,"Suge Knight, Katt Williams, ordered to trial i...",SUBSCRIBE FOLLOW US © 2015 Cox Media Group . B...,"['subscribe', 'follow', 'us', 'two thousand an..."


In [1]:
%%time

def load_vectors(fname):
    import io
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return n,d,data

# def load_vectors(fname):
#     with open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore') as fin:
#         n, d = map(int, fin.readline().split())
#         data = {}
#         for line in fin:
#             tokens = line.rstrip().split(' ')
#             data[tokens[0]] = list(map(float, tokens[1:]))
#     return n,d,data

n,d,ft_dict = load_vectors("assets/wiki-news-300d-1M.vec")

CPU times: user 2min 28s, sys: 1min 7s, total: 3min 36s
Wall time: 4min 42s


In [8]:
import numpy as np
def get_ft_vec(token,ft_dict,d=300):
    try:
        v = np.array(ft_dict[token])
    except KeyError:
        v = np.array([0]*d)
    return v

In [9]:
'''Most important section : The way the vectors are created!'''

def compute_ft_sum(textlist,ft_dict=ft_dict,d=300):
    ret_vec = np.array([0]*d)
    for key in textlist:
        vec = get_ft_vec(key,ft_dict)
        # Vector addition of token embeddings
        ret_vec = ret_vec + vec
    return list(ret_vec)


In [10]:
%%time
# (df.head())
# df.text
from multiprocessing import Pool

with Pool() as pool:
    results = pool.map(compute_ft_sum, list(df.pre_processed_text))

CPU times: user 1min 8s, sys: 1min 24s, total: 2min 32s
Wall time: 52min 4s


In [11]:
thattable = pd.DataFrame(results)

In [12]:
thattable["label"] = df.label

In [16]:
thattable.shape

(383663, 301)

In [13]:
thattable.to_csv("Data/Training_Data/alldata_embedded_vector.csv",index=False)

In [14]:
def norm_to_unit_vector(df):
    sq = np.square(df.drop("label", axis=1))
    sqrrt = np.sqrt(sq.sum(axis=0))

    final = df.drop("label", axis=1).divide(sqrrt, axis=1)
    return final

In [15]:
df_norm = norm_to_unit_vector(thattable)

In [17]:
df_norm["label"] = thattable["label"]

In [18]:
df_norm.to_csv("Data/Training_Data/alldata_embedded_vector_norm.csv",index=False)